<a href="https://colab.research.google.com/github/Anas-art-source/AI_experiments/blob/main/Unsloth_gemma_2b_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
!pip install -qqq -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "google/gemma-2b", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [ ]:
import pandas as pd
from datasets import load_dataset

df = pd.read_json('augment_book_qa.json')
df.head()

,question,previous_answer,my_answer
0,What are the different types of frequency mome...,The three types of frequency moments of data s...,**Types of Frequency Moments of Data Streams**...
1,What is the k-Means Clustering algorithm and w...,The k-Means Clustering algorithm is a method f...,The k-Means Clustering algorithm is a widely u...
2,What is spectral clustering and how is it used...,Spectral clustering is a technique used in clu...,**What is Spectral Clustering?**\n\nSpectral c...
3,What are the reasons for the fundamental chang...,The reasons for the fundamental change in comp...,"I apologize, but the provided context does not..."
4,In which decade did computer science begin as ...,Computer science began as an academic discipli...,Computer science emerged as an academic discip...


In [ ]:
df = df.rename(columns={'question': 'prompt', 'previous_answer': 'regected', 'my_answer': 'chosen'})


In [ ]:
df_train = df.iloc[:-300]
df_test = df.iloc[-300:]

In [ ]:
df_train.shape, df_test.shape

((1096, 3), (300, 3))

In [ ]:
from datasets import Dataset, DatasetDict

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })
datasets_train_test.push_to_hub("Anas989898/DPO-datascience")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Anas989898/DPO-datascience/commit/4d2bebb7859c7ed8442a39d6eaa5f77dcef70a41', commit_message='Upload dataset', commit_description='', oid='4d2bebb7859c7ed8442a39d6eaa5f77dcef70a41', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
alpaca_prompt = """

### Question:
{}

### Response:
{}

"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["prompt"]
    answer      = examples["chosen"]
    texts = []
    for question, answer in zip(question, answer):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = datasets_train_test.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'regected', 'chosen', 'text'],
        num_rows: 1096
    })
    test: Dataset({
        features: ['prompt', 'regected', 'chosen', 'text'],
        num_rows: 300
    })
})

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1096 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,096 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 9,805,824


Step,Training Loss
1,1.241400
2,1.257600
3,1.300600
4,1.303100
5,1.291400
6,1.354100
7,1.040200
8,1.019300
9,1.094700
10,1.130700


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Calculate the precision for me when the TP is 30 and FP is 130.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>\n\n### Question:\nCalculate the precision for me when the TP is 30 and FP is 130.\n\n### Response:\n\n\nThe precision of a classifier is a measure of how accurately it identifies the true positive cases. It is calculated as the ratio of the number of true positives (TP) to the total number of predicted positives (TP + FP), where TP represents the number of actual positive cases correctly classified as positive, and FP represents the number of false positives (falsely classified as positive).\n\nIn this case, the TP is 30, and the FP is 130. To calculate the precision, we divide the number of true positives by the total number of predicted positives:\n\nPrecision = TP / (TP + FP) = 30 / (30 + 130) = 30 / 160 = 0.1875\n\nTherefore, the precision for this classifier is 0.1875, which means that the classifier correctly identifies 18.75% of the true positive cases.\n\n<eos>']

In [ ]:
model.save_pretrained("lora_model") # Local saving


In [ ]:
model.save_pretrained_merged("google/gemma-2b", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("Anas989898/gemma_2b_it_ds", tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.4 out of 12.67 RAM for saving.


100%|██████████| 18/18 [00:00<00:00, 29.68it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving google/gemma-2b/pytorch_model-00001-of-00002.bin...
Unsloth: Saving google/gemma-2b/pytorch_model-00002-of-00002.bin...


Unsloth: You are pushing to hub, but you passed your HF username = Anas989898.
We shall truncate Anas989898/gemma_2b_it_ds to gemma_2b_it_ds


...

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/Anas989898/gemma_2b_it_ds
